In [2]:
import pandas as pd
import json

#first 2 million
data = []
ncount = 0
with open('reviews_Electronics.json') as f:
    for line in f:
        ncount += 1
        if ncount <= 10000:
            if ncount%100000 == 0:
                print(ncount)
            data.append(json.loads(line))

df = pd.DataFrame(data)

In [3]:
del df["unixReviewTime"]
del df["reviewTime"]
del df["reviewerName"]
del df["reviewerID"]
del df["helpful"]

In [4]:
df.head(5)

,asin,overall,reviewText,summary
0,0132793040,5.0,Corey Barker does a great job of explaining Bl...,Very thorough
1,0321732944,5.0,While many beginner DVDs try to teach you ever...,Adobe Photoshop CS5 Crash Course with master P...
2,0439886341,1.0,It never worked. My daughter worked to earn th...,absolutely horrible
3,0439886341,3.0,Some of the functions did not work properly. ...,Disappointing
4,0439886341,1.0,Do not waste your money on this thing it is te...,TERRIBLE DONT WASTE YOUR MONEY


In [5]:
df["overall"] = df["overall"].astype(int)

In [6]:
df = df[df["overall"] != 3]
df["sentiment"] = df["overall"] >= 4

In [7]:
import numpy as np
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm

train, test = cross_validation.train_test_split(df, test_size = 0.2)

In [8]:
df.index = range(0,len(df))
train.index = range(0,len(train))
test.index = range(0,len(test))

train.head()

,asin,overall,reviewText,summary,sentiment
0,9573212919,2,Sometimes cheap is good. You go a long way wit...,"Cheap, malfunctioning machine.",False
1,B00000DM9W,5,After researching shower radios and looking at...,Great Fit and Finish for Shower,True
2,1400599997,5,"This is a good item. Even though it's old, it'...",Still a good product,True
3,9966338926,2,Original Review: This cable does the job but b...,"Not so good cable, 40"" long - update",False
4,8805002097,5,These lights work great. They do the job perf...,Great lights!,True


In [9]:
from bs4 import BeautifulSoup 
import re
import nltk
nltk.download()
from nltk.corpus import stopwords

def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [10]:
num_reviews = train["reviewText"].size

print("Cleaning and parsing the training set product reviews...\n")
clean_train_reviews = []
for i in range( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print("Review %d of %d\n" % ( i+1, num_reviews ))                                                                    
    clean_train_reviews.append( review_to_words( train["reviewText"][i] ))

Cleaning and parsing the training set product reviews...



/Applications/anaconda/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Review 1000 of 7380

Review 2000 of 7380

Review 3000 of 7380

Review 4000 of 7380

Review 5000 of 7380

Review 6000 of 7380

Review 7000 of 7380



In [11]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Creating the bag of words...



In [12]:
vocab = vectorizer.get_feature_names()
print(vocab)

['aa', 'aaa', 'abilities', 'ability', 'able', 'absolute', 'absolutely', 'absurd', 'abuse', 'abused', 'ac', 'accept', 'acceptable', 'accepted', 'accepts', 'access', 'accessed', 'accessible', 'accessing', 'accessories', 'accessory', 'accident', 'accidental', 'accidentally', 'accidently', 'accommodate', 'accomodate', 'accomplish', 'according', 'account', 'accounts', 'accuracy', 'accurate', 'accustomed', 'acer', 'acknowledge', 'acquired', 'across', 'act', 'acting', 'action', 'activate', 'activated', 'active', 'activity', 'acts', 'actual', 'actually', 'ad', 'adapt', 'adapter', 'adapters', 'adaptor', 'adaptors', 'add', 'added', 'adding', 'addition', 'additional', 'additionally', 'address', 'addressed', 'addresses', 'adds', 'ade', 'adequate', 'adequately', 'adjust', 'adjustable', 'adjusted', 'adjusting', 'adjustment', 'adjustments', 'adjusts', 'admit', 'admittedly', 'adobe', 'ads', 'adult', 'adults', 'advance', 'advanced', 'advantage', 'advantages', 'advent', 'adventure', 'adventures', 'adver

In [13]:
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
counted = zip(vocab, dist)
for tag, count in sorted(counted, key=lambda counted: counted[1], reverse=True):
    print(count, tag)

3590 nook
2719 one
2549 great
2147 use
2038 would
1872 good
1784 like
1684 product
1665 get
1463 works
1406 books
1400 well
1306 price
1306 tv
1301 bought
1287 kindle
1244 time
1212 buy
1191 work
1187 easy
1162 screen
1153 radio
1127 read
1110 also
1038 cable
1024 even
1000 much
995 tablet
959 phone
945 love
933 quality
912 really
895 back
864 used
836 still
831 card
819 got
818 battery
812 book
801 mount
792 new
791 device
775 need
772 better
762 want
758 could
740 color
737 amazon
736 recommend
726 little
717 reading
701 two
700 first
679 using
652 problem
651 sound
648 reader
641 find
637 purchased
630 go
611 way
611 worked
609 touch
606 wall
587 best
582 case
570 store
567 since
566 thing
559 able
549 nice
547 another
546 make
539 without
529 right
527 item
527 purchase
523 put
517 many
510 long
506 unit
499 money
498 know
497 never
494 take
489 computer
483 free
478 fine
472 made
469 barnes
469 charge
469 cover
468 player
464 found
462 problems
461 old
461 see
458 lot
456 usb
451 

In [14]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

Training the random forest...


In [15]:
# Create an empty list and append the clean reviews one by one
num_reviews = len(test["reviewText"])
clean_test_reviews = [] 

print("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["reviewText"][i] )
    clean_test_reviews.append( clean_review )

Cleaning and parsing the test set movie reviews...



/Applications/anaconda/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Review 1000 of 1845



In [16]:
# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"asin":test["asin"], "predicted_sentiment":result} )

output.head(20)

,asin,predicted_sentiment
0,1400532655,False
1,B00000J08Q,True
2,8862935307,True
3,8862935293,True
4,9984984354,True
5,1400532655,True
6,B00000DM9R,True
7,1400532655,False
8,1400532655,True
9,B00000J0E8,False


In [17]:
test["predicted_sentiment"] = output["predicted_sentiment"]
test["modelCorrect"] = test["sentiment"] == test["predicted_sentiment"]
test_score = (test["modelCorrect"] == 1).sum() / test["modelCorrect"].count()
print(test_score)

0.874796747967


/Applications/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Applications/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
